### Algoritmo que realiza las siguientes funciones:
    1. Abrir un fichero Excel (.xls) como si fuera un (.txt), para obtener un (.xml).
    2. Parsear el contenido del (.xml) y obtener toda la información del Excel(todas las celdas).
        2.1. Se obtiene toda la información de cada fila del Excel -> ROW.
        2.2. Se obtiene toda la información de cada celda del Excel -> CELL. 
            2.2.1. Se obtienen los valores MergeDown y MergeAcross de cada celda.
        2.3. Se obtiene la información "visual"  de cada celda -> DATA.
    3. Con toda la información anterior, se crea un fichero (.csv) y se va guardando la información correctamente.
    
#### Autor: Mario de la Parte Izquierdo 
#### Fecha: 31/03/2019

#### Cambio 12/06/2019: Permitir introducir datos a la BBDD a partir de un fichero Excel con múltiples titulaciones.
Se decide modificar el fichero (.csv) final, añadiendo a la cabecera principal de los datos, una nueva columna
llamada "Plan" e introduciendo el Plan o titulación de cada conjunto de datos. De esta forma, se pueden eliminar las cabeceras 
que se repetían en el Excel (.xls) original (en el caso de tener más de una titulación por Excel.

In [70]:
import re # Regular Expressions

In [1]:
f = open("datos.xls","r") # Se abre el fichero original corrupto [r: read(estamos abriendo un archivo para leerlo)].
a = f.read() 
f.close()
#print(a) # Visualizar todo lo que tiene el xml.

In [81]:
nombreDelArchivo = "ficheroBueno.csv" # Variable global que recoge el nombre del archivo .csv que se va a crear.
archivo = open(nombreDelArchivo ,"w")  # 1. Se crear el fichero (.csv)

posicion = 0
lista_num_ceros_fila_siguiente = []
filaMergeDownMayorQueCero = False
filaEspecial = False
flag = True
Responsable = False
Plan = ""
hacerUnaVezSolo = True
contadorFila = 0


datos = re.findall("<Row>(.+?)</Row>", a) # Se obtiene toda la información de cada fila del Excel (ROW)

for i, fila in enumerate(datos): # Se recorre cada fila
    #print(i)
    #print(fila)
    #print("\n")
    # Conclusión: Hay que tratar 21 filas    
    cell = re.findall("<Cell(.+?)</Cell>", fila) # CELL (es una lista de strings)
    
    
    #print("(inicio)--------------------------")
    #print(cell)
    #print("contadorFila = "+ str(contadorFila) +"--------------------------(fin)")
    contadorFila +=1
    
    #@
    primeraCeldaDeLaFilaEsNumDe4Dig = False
    
    for i, celd in enumerate(cell): 
        data = re.findall("<Data(.+?)>(.+?)</Data>", celd)
        print(data)
        if (len(data) >0 ):
            if ( (i == 0) and (len(data[0][1]) == 4) and (data[0][1] != "Rep.")):  # and (type(data[0][1]) == int)
                #print("Longitud de 4") 
                print("len de data: "+str(len(data[0][1])))
                primeraCeldaDeLaFilaEsNumDe4Dig = True
        
        if(len(data)>0):
            Plan234 = data[0][1]  # Variable para      
            if (  len(data[0][1] ) > 40 and  Plan234.find("Plan") == 0 ):
                Plan = data[0][1] # Se almacena el plan a introducir 
        
    #@
    
    if ( contadorFila <= 6 or primeraCeldaDeLaFilaEsNumDe4Dig == True ): # Si cumple estas condiciones, se añade al csv final.
        print("************************************************************************")
        print("Fila número "+str(contadorFila))
        
        for j, celda in enumerate(cell):
            step_0 = celda.split(' ss:MergeDown="')  # Se saca MergeDown y  MergeAcross de cada celda
            step_1 = step_0[1].split('" ss:MergeAcross="')
            step_2 = step_1[1].split('"')
            #print("MergeDown:")
            #print(step_1[0])
            MergeDown = step_1[0] 
            #print("MergeAcross:")
            #print(step_2[0])
            MergeAcross = step_2[0]

            # Se rellena la lista_num_ceros_fila_siguiente 
            if(int(MergeDown) > 0):
                filaMergeDownMayorQueCero = True
            if (filaMergeDownMayorQueCero == True):
                lista_num_ceros_fila_siguiente.append(int(MergeDown)) # Para añadir MergeDown a la lista sea 1 o 0.
                if(int(MergeAcross) > 0):
                    aux = int(MergeAcross)
                    while (aux > 0):
                        lista_num_ceros_fila_siguiente.append(0) # Para añadir un cero en caso de que haya celdas en blanco
                        aux -= 1

            if (filaEspecial == True and hacerUnaVezSolo == True): # Meter info de lista_num_ceros_fila_siguiente [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1]
                #print("filaEspecial activado")

                #if (len(lista_num_ceros_fila_siguiente) >= posicion):# Para limpiar la lista en caso de que haya 2 titulaciones en el mismo Excel.
                    #lista_num_ceros_fila_siguiente = []
                    #posicion = 0
                #print(len(lista_num_ceros_fila_siguiente))
                while (posicion < len(lista_num_ceros_fila_siguiente)   and flag == True):
                    #if (hacerUnaVezSolo == True):
                        if (lista_num_ceros_fila_siguiente[posicion] == 1): # Hay un 1 en la lisya -> meter ; 
                            #print("mete celda en blanco")
                            archivo.write(";")
                        else:                                           # Hay un 0 en la lisya -> meter datos 
                            flag = False
                            #print("mete datos")
                            data = re.findall("<Data(.+?)>(.+?)</Data>", celda) # DATA 
                            #print(data)
                            if(len(data)>0):
                                if ( type(data[0][1]) != int ): # Si no es un entero, se entrecomilla
                                    archivo.write('"') # Se añaden comillas para que se posteriormente se pueda abrir el .csv sin problemas por lasa tildes
                                    archivo.write(data[0][1])
                                    archivo.write('"') # Se añaden comillas para que se posteriormente se pueda abrir el .csv sin problemas por lasa tildes
                                    archivo.write(";")
                                else:  # si es un entero, no se entrecomilla
                                    print("es un entero") # Aqui no entra nunca 
                                    archivo.write(data[0][1])
                                    archivo.write(";") # Siguiente columna
                                num = int(MergeAcross)
                                while(num > 0):            
                                    archivo.write(";")
                                    num -= 1
                            else: 
                                archivo.write(";")
                        posicion += 1  
                #hacerUnaVezSolo = False        
            else:   # Caso Normal         
                # Sacar contenido de la celda (DATA)
                data = re.findall("<Data(.+?)>(.+?)</Data>", celda) # DATA 
                if(len(data)>0):
                    # AKI
                    print(data[0][1])

                    #if (variable.find("Plan "+[0-9]) < 0):
                    Plan234 = data[0][1]  # Variable para      
                    if (  len(data[0][1] ) > 40 and  Plan234.find("Plan") == 0 ):
                        print("la cadena anterior es larga y empieza por plan___________________________________________________")
                        Plan = data[0][1] # Se almacena el plan a introducir 

                    if ( str(data[0][1]) == "Responsable" ):
                        Responsable = True
                        #print("Después de este habría que meter la nueva columna")


                    if ( type(data[0][1]) != int ): # Si no es un entero, se entrecomilla
                        archivo.write('"') # Se añaden comillas para que se posteriormente se pueda abrir el .csv sin problemas por lasa tildes
                        archivo.write(data[0][1])
                        archivo.write('"') # Se añaden comillas para que se posteriormente se pueda abrir el .csv sin problemas por lasa tildes
                        archivo.write(";") # Siguiente columna
                    else:  # si es un entero, no se entrecomilla
                        print("es un entero") # Aqui no entra nunca 
                        archivo.write(data[0][1])
                        archivo.write(";") # Siguiente columna
                    num = int(MergeAcross)
                    while(num > 0):            
                        archivo.write(";")
                        num -= 1
                else: # Es una celda en blanco (en data hay [])
                    #print("EN BLANCO")
                    archivo.write(";")

            flag = True

        #print(lista_num_ceros_fila_siguiente)
        # Fin del 2do for que recorre cada fila.
        #print("\n") # Por cada cacho del sublime (21 en total)

        if (filaEspecial == True and hacerUnaVezSolo == True):
            while (posicion < len(lista_num_ceros_fila_siguiente) ):
                archivo.write(";") 
                posicion += 1 
            archivo.write(";") # Se añade otra celda en blanco (la celda de debajo de la nueva columna "Plan")
            hacerUnaVezSolo = False 

        #AKI (Antes de hacer el \n de "Responsable")
        if (Responsable == True): # Para insertar la nueva columna llamada "Plan"
            archivo.write('"') 
            archivo.write("Plan")
            archivo.write('"') 
            archivo.write(";") 
            Responsable = False

        if(contadorFila > 6): # Se introduce la info relevante a la nueva columna "Plan" añadida
            archivo.write(Plan)
            archivo.write(";")
            archivo.write("\n") 
        else: 
            archivo.write("\n") # Siguiente Fila.

        filaEspecial = False
        if(filaMergeDownMayorQueCero == True):
            filaEspecial = True
        filaMergeDownMayorQueCero = False
    
archivo.close()
#print(lista_num_ceros_fila_siguiente) # Son 33 columnas

[(' ss:Type="String"', 'Control de Grupos de Matrícula')]
************************************************************************
Fila número 1
Control de Grupos de Matrícula
[(' ss:Type="String"', 'Año académico:')]
[(' ss:Type="String"', '2018/19-0')]
[(' ss:Type="String"', 'Fecha de creación:')]
[(' ss:Type="String"', '29/03/2019')]
[(' ss:Type="String"', 'Centro (desde):')]
[(' ss:Type="String"', '13 - Escuela Politécnica Superior')]
[(' ss:Type="String"', 'Centro (hasta):')]
[(' ss:Type="String"', '13 - Escuela Politécnica Superior')]
[(' ss:Type="String"', 'Plan (desde):')]
[(' ss:Type="String"', '263 - MÁSTER UNIVERSITARIO EN INTELIGENCIA DE NEGOCIO Y BIG DATA EN ENTORNOS SEGUROS')]
[(' ss:Type="String"', 'Plan (hasta):')]
[(' ss:Type="String"', '264 - MÁSTER UNIVERSITARIO EN INGENIERÍA INFORMÁTICA (PLAN 2018)')]
[(' ss:Type="String"', 'Tipología académica:')]
[(' ss:Type="String"', 'Todas')]
************************************************************************
Fila número 2

[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '7')]
[(' ss:Type="Number"', '1')]
[(' ss:Type="String"', 'Manual')]
[]
[]
[(' ss:Type="Number"', '4772')]
[(' ss:Type="Number"', '1016331')]
[(' ss:Type="String"', 'GONZALEZ MARTIN, ROCIO')]
[(' ss:Type="String"', 'S')]
[(' ss:Type="String"', 'N')]
[(' ss:Type="String"', 'S')]
[(' ss:Type="String"', 'N')]
************************************************************************
Fila número 32
8106
INTELIGENCIA NEGOCIO APLICADA I
1
90
Teoría
S
S
2
Mixto
0
27
0
0
27
0
7
20
0
0
0
0
7
1
Manual
4772
1016331
GONZALEZ MARTIN, ROCIO
S
N
S
N
[(' ss:Type="Number"', '8106')]
len de data: 4
[(' ss:Type="String"', 'INTELIGENCIA NEGOCIO APLICADA I')]
[(' ss:Type="Number"', '1')]
[(' ss:Type="Number"', '90')]
[(' ss:Type="String"', 'Teoría')]
[(' ss:Type="String"', 'S')]
[(' ss:Type="String"', 'S')]
[(' ss:Type="Number"', '2')]
[(' ss:Type="String"', 'Mixto')]
[('

[(' ss:Type="String"', 'S')]
[(' ss:Type="Number"', '1')]
[(' ss:Type="String"', 'Mixto')]
[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '15')]
[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '15')]
[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '12')]
[(' ss:Type="Number"', '3')]
[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '12')]
[(' ss:Type="Number"', '1')]
[(' ss:Type="String"', 'Manual')]
[]
[]
[(' ss:Type="Number"', '1595')]
[(' ss:Type="Number"', '90225')]
[(' ss:Type="String"', 'Gonzalez Moya, Francisco Javier')]
[(' ss:Type="String"', 'S')]
[(' ss:Type="String"', 'S')]
[(' ss:Type="String"', 'S')]
[(' ss:Type="String"', 'N')]
************************************************************************
Fila número 58
8115
SISTEMAS DE INFORMACIÓN AVANZADOS
1
90
Teoría
S
S
1
Mixto
0
15
0
0
15
0
12
3
0
0
0
0
12
1
Manual
1595
90225
Gonzalez Moya, Fra

[(' ss:Type="String"', 'N')]
************************************************************************
Fila número 80
8121
TRABAJO FIN DE MÁSTER
1
91
Teoría
S
S
2
Mixto
0
10
0
0
10
0
5
5
0
0
0
0
5
1
Manual
1858
1000505
Saiz Diez, Jose Manuel
S
N
N
N
[(' ss:Type="Number"', '8121')]
len de data: 4
[(' ss:Type="String"', 'TRABAJO FIN DE MÁSTER')]
[(' ss:Type="Number"', '1')]
[(' ss:Type="Number"', '91')]
[(' ss:Type="String"', 'Teoría')]
[(' ss:Type="String"', 'S')]
[(' ss:Type="String"', 'S')]
[(' ss:Type="Number"', '2')]
[(' ss:Type="String"', 'Mixto')]
[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '10')]
[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '10')]
[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '5')]
[(' ss:Type="Number"', '5')]
[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '0')]
[(' ss:Type="Number"', '5')]
[(' ss:Type="Number"', '1')]
[(' ss:Type="String"', 'Manual')]
